In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.tokenization_utils_base import BatchEncoding

In [ ]:
dataset = load_dataset('imdb')

In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
MODEL_NAME = 'microsoft/deberta-v3-small'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
MAX_LENGTH = 512

def preprocess_train(sample):
    return tokenizer(sample['text'], truncation=True, max_length=128)


In [ ]:
ds_train = dataset['train'].map(preprocess_train).remove_columns(['text'])

In [ ]:
collator = DataCollatorWithPadding(tokenizer)
def collate_fn(batch):
    return collator(batch)  

train_loader = torch.utils.data.DataLoader(
    ds_train,
    batch_size=4,
    num_workers=0,
    drop_last=True,
    collate_fn=collate_fn,
)

In [ ]:
for x in train_loader:
    print(x['input_ids'].size())
    break

In [ ]:
device = 'cuda:0'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters())
optimizer.zero_grad()

In [ ]:
for x in train_loader:
    #model_inputs = BatchEncoding(x).to(device)
    for k in x:
        x[k] = x[k].to(device)
    outputs = model(**x)
    print(outputs.loss.item())
    
    outputs.loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    

In [ ]:
outputs